<a href="https://colab.research.google.com/github/osamatarabih/Optimal_BMPs_Assign/blob/main/LOW_OptBMPs_Application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/FEC_Opt_Scens/BMPs_1500000bound_N_2549.099.txt', sep=',')
LOOKUP_Table = pd.read_csv('/content/drive/MyDrive/FEC_Opt_Scens/LOOKUP_Table.csv')
LU_Base = df[' BMPs'].apply(lambda x: LOOKUP_Table[(LOOKUP_Table['BMP_Type'] == x)]['LU_Base'].values[0])
LU_BMP = df[' BMPs'].apply(lambda x: LOOKUP_Table[(LOOKUP_Table['BMP_Type'] == x)]['LU_Code'].values[0])
Out_df = pd.DataFrame()
Out_df['Subbasin'] = df['Node']
Out_df['BMP'] = df[' BMPs']
Out_df['LU_Base'] = LU_Base
Out_df['LU_BMP'] = LU_BMP

Out_df.to_csv('/content/drive/MyDrive/FEC_Opt_Scens/BMPs_1500000bound_N_2549.099.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Import required Packages
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

# read the existing LU with Subbasins
Existing_LU = pd.read_csv('/content/drive/MyDrive/FEC_Opt_Scens/FEC_Landuse_Subbasin.csv')

Optimized_LU = pd.read_csv('/content/drive/MyDrive/FEC_Opt_Scens/BMPs_1500000bound_N_2549.099.csv')

Opt_LU = np.zeros(len(Existing_LU))
Opt_BMP = np.zeros(len(Existing_LU))
for i in range (len(Existing_LU)):
    for j in range (len(Optimized_LU)):
            if Existing_LU['Subbasin'].iloc[i] == Optimized_LU['Subbasin'].iloc[j]:
                if Existing_LU['LUID'].iloc[i] == Optimized_LU['LU_Base'].iloc[j] or (Existing_LU.duplicated(['OBJECTID'], keep=False)[i] == True and Existing_LU.loc[Existing_LU['OBJECTID'] == Existing_LU['OBJECTID'].iloc[i]]['LUID'].any() ==  Optimized_LU['LU_Base'].iloc[j]):
                    Opt_LU[i] = Optimized_LU['LU_BMP'].iloc[j]
                    Opt_BMP[i] = Optimized_LU['BMP'].iloc[j]
                    break
                else:
                    Opt_LU[i] = Existing_LU['LUID'].iloc[i]
                    Opt_BMP[i] = 0
            else:
                Opt_LU[i] = Existing_LU['LUID'].iloc[i]
                Opt_BMP[i] = 0
Existing_LU['Opt_LU'] = Opt_LU
Existing_LU['Opt_BMP'] = Opt_BMP

# Now the LU with Optimized LUIDs include many LUs that are separated between various subbasins, but those separated LUs need to be
# aggregated into One LU again, I wil do this through delete the replicates based on the OBJECTID (based on the original LUs data)
Existing_LU = Existing_LU.drop_duplicates(subset='OBJECTID')
# Existing_LU.to_csv('C:/Osama_PC/LOW_WAM/Analyses/UK/Opt_Val/LU_with_OptimizedBMPs.csv')

#### Now Add Optimized LUID into the LandUse File
#Read the Landuse File
Landuse = pd.read_csv('/content/drive/MyDrive/FEC_Opt_Scens/FEC_Base_LU.csv')
Landuse = Landuse.set_index('OBJECTID')
for i in Landuse.index:
    if not i in Existing_LU['OBJECTID'].unique():
        d = {'OBJECTID':i,'LUID':Landuse.loc[i]['LUID'],'Opt_LU':Landuse.loc[i]['LUID'],'Subbasin':0}
        df = pd.DataFrame(data=d,index=[0])
        Existing_LU = pd.concat([Existing_LU,df])
Existing_LU.to_csv('/content/drive/MyDrive/FEC_Opt_Scens/FEC_1500000bound_N_2549.099.csv')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
